In [1]:
import os
import vertexai
from vertexai.generative_models import GenerativeModel, Part
from google.cloud import storage
from PyPDF2 import PdfReader, PdfWriter

In [2]:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "ui-ux-model-6e8765c9a1a6.json"

In [3]:
project_id = "ui-ux-model"
location="us-central1"
model_name = "gemini-1.5-flash-preview-0514"

In [4]:
vertexai.init(project=project_id, location=location)

In [5]:
model = GenerativeModel(model_name=model_name)

In [6]:
# Set up Cloud Storage client
storage_client = storage.Client()
bucket_name = 'test-pdfs-store'
input_folder = 'input_files'
output_folder = 'output_files'

In [24]:
# Get the bucket
bucket = storage_client.get_bucket(bucket_name)

In [8]:
# List all the PDF files in the input folder
blobs = list(bucket.list_blobs(prefix=f"{input_folder}/"))
pdf_files = [blob.name.split('/')[-1] for blob in blobs if blob.name.endswith('.pdf')]

In [9]:
# Send the request to the vertex ai
def send_request_to_vertex_ai(pdf_file_uri):
    prompt = """
    Analyze the PDF and provide the page breaks for each chapter.
    Return the result as a JSON object with the following format:
    {
      "page_breaks": [1, 4, 6]
    }
    """
    
    pdf_to_analyse = Part.from_uri(pdf_file_uri, mime_type="application/pdf")
    contents = [pdf_to_analyse, prompt]
    return model.generate_content(contents)

In [49]:
# Split the PDF document based on the extracted page breaks
def split_pdf_and_save_in_bucket(page_breaks, pdf_file):
    if page_breaks:
        blob = bucket.blob(pdf_file)
        print("Blob: ", blob)
        blob.reload()  # or blob.load_from_dict({})
        metadata = blob.metadata
        print(f"Metadata: {metadata}")
        destination_filename = f"output_files/{pdf_file}"
        print(destination_filename)
        blob.download_to_filename(destination_filename)

        with open("downloaded_document.pdf", "wb") as file:
            file.write(content)
        
        with blob.open('rb') as file:
            pdf_reader = PdfReader(file)
            pdf_writer = PdfWriter()
            start = 0
            for end in page_breaks:
                for page_num in range(start, end):
                    pdf_writer.add_page(pdf_reader.pages[page_num])
                output_path = f"{output_folder}/{pdf_file_uri.split('/')[-1]}_part_{end//5}.pdf"
                with open(output_path, 'wb') as output_file:
                    pdf_writer.write(output_file)
                
                # Upload the split PDF file to the GCS bucket
                blob = bucket.blob(f"{output_folder}/{pdf_file_uri.split('/')[-1]}_part_{end//5}.pdf")
                blob.upload_from_filename(output_path)
                print(f"Saved {pdf_file_uri.split('/')[-1]}_part_{end//5}.pdf to {bucket_name}")
                
                pdf_writer = PyPDF2.PdfWriter()  # Reset the writer for the next part
                start = end
        
        print("PDF split and uploaded successfully.")

In [50]:
# Main Function
for pdf_file in pdf_files:
    pdf_file_uri = f"gs://{bucket_name}/{input_folder}/{pdf_file}"
    # response = send_request_to_vertex_ai(pdf_file_uri)
    # print("Response: ", response)
    page_breaks = [1, 3, 8]
    split_pdf_and_save_in_bucket(page_breaks, pdf_file)

Blob:  <Blob: test-pdfs-store, Chapter-4.pdf, None>


NotFound: 404 GET https://storage.googleapis.com/storage/v1/b/test-pdfs-store/o/Chapter-4.pdf?projection=noAcl&prettyPrint=false: No such object: test-pdfs-store/Chapter-4.pdf